<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=63cc19f2d751ab79d9f16a38cec89d00a52a769a2a8fa61c9dbe4c5f6032d13f
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2


    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-08 10:21:49
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.40 C
-------------------
Today PnL: -87.15 K (-0.62%)
Current PnL: -27.78 L (-17.94%)
CY Booked + Current PnL: -13.49 L (-8.71%)
-------------------
Total profit:  1.43 L
Total loss:  -29.21 L
-------------------
Total Booked + Current PnL: 13.49 L (10.55%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.17%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 92.41 L (65.77%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 5.33%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.13,-16.73,20.20,0.09,16952.0,-16858.0,83919.0,94.88,38.0,M-SC,2.38,253.0,-0.99,0.62,10.25,OX40N,NTT,DURABLES
50,MASFIN,398.61,2.71,0.83,21.05,22.05,20795.0,810.0,98790.0,-13.28,63.0,H-SC,7.10,164.0,0.04,0.73,43.60,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-0.30,-12.50,14.30,0.02,21842.0,-21816.0,152743.0,132.70,52.0,M-SC,16.87,234.0,-1.00,1.13,32.86,OX40N,NTT,PAINTS
43,ITC,452.00,-0.20,-1.25,11.84,10.44,23399.0,-2509.0,197629.0,-41.15,48.0,X-LC,1.57,5.0,-0.11,1.46,4.24,X40,NTT,FMCG
35,ICICIGI,2252.93,-0.58,4.72,14.78,20.19,26981.0,8230.0,182550.0,-17.91,43.0,X-MC,6.16,68.0,0.31,1.35,20.55,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,398.61,2.71,0.83,21.05,22.05,20795.0,810.0,98790.0,-13.28,63.0,H-SC,7.10,164.0,0.04,0.73,43.60,XR,ATH,FINANCE
68,STARHEALTH,761.00,1.59,-9.99,63.60,47.26,160935.0,-28081.0,253042.0,22.87,37.0,H-SC,13.81,171.0,-0.17,1.87,35.57,XY24,NTT,INSURANCE
13,BSOFT,831.70,1.31,-20.52,89.71,50.79,100289.0,-28861.0,111792.0,2.05,74.0,H-SC,5.64,151.0,-0.29,0.82,30.16,XR,ATH,IT
36,ICICIPRULI,790.00,0.99,4.82,24.95,30.97,46220.0,8511.0,185249.0,-19.89,61.0,X-MC,2.24,75.0,0.18,1.37,17.98,X40,ATH,INSURANCE
56,RAJESHEXPO,518.00,0.95,-63.99,177.90,0.07,87875.0,-87781.0,49396.0,1710.61,50.0,L-SC,2.35,268.0,-1.00,0.36,22.85,OX40N,NTT,JEWELLERY


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
10,BANDHANBNK,400.0,-4.13,-32.62,185.55,92.42,348290.0,-90852.0,187707.0,32.18,25.0,H-SC,7.02,170.0,-0.26,1.38,8.48,XY24,NTT,BANKS
18,COFFEEDAY,80.0,-3.84,-45.79,149.53,35.27,92043.0,-51994.0,61555.0,-57.75,21.0,L-SC,12.92,270.0,-0.56,0.45,50.66,XR,NTT,HOTELS
80,VALIANTORG,838.0,-3.65,-46.18,223.55,74.15,196858.0,-75545.0,88060.0,-258.45,38.0,H-SC,20.90,149.0,-0.38,0.65,12.88,XR,NTT,CHEMICALS
49,LAOPALA,464.0,-2.01,-37.52,122.62,39.09,100442.0,-49192.0,81913.0,79.36,29.0,H-SC,6.84,155.0,-0.49,0.60,5.50,AR,NTT,CERAMICS
83,WHIRLPOOL,2270.0,-1.98,-25.41,143.90,81.92,123213.0,-29174.0,85624.0,-56.34,19.0,M-SC,8.13,236.0,-0.24,0.63,1.37,XR,NTT,DURABLES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,398.61,2.71,0.83,21.05,22.05,20795.0,810.0,98790.0,-13.28,63.0,H-SC,7.1,164.0,0.04,0.73,43.6,XR,ATH,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-1.15,-7.78,125.46,107.91,170820.0,-11489.0,136155.0,-25.01,27.0,M-SC,11.56,216.0,-0.07,1.00,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.30,-12.50,14.30,0.02,21842.0,-21816.0,152743.0,132.70,52.0,M-SC,16.87,234.0,-1.00,1.13,32.86,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.13,-16.73,20.20,0.09,16952.0,-16858.0,83919.0,94.88,38.0,M-SC,2.38,253.0,-0.99,0.62,10.25,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.75,-24.44,50.18,13.47,102247.0,-65907.0,203760.0,-68.90,29.0,H-SC,2.84,158.0,-0.64,1.50,3.61,XY24,NTT,PAINTS
66,SIS,528.00,-1.20,-25.34,62.41,21.26,51738.0,-28132.0,82900.0,1954.40,43.0,H-SC,4.23,166.0,-0.54,0.61,12.01,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.50,7.14,60.78,72.27,110345.0,12100.0,181548.0,-7.89,74.0,M-LC,2.79,99.0,0.11,1.34,13.76,XR,NTT,IT
50,MASFIN,398.61,2.71,0.83,21.05,22.05,20795.0,810.0,98790.0,-13.28,63.0,H-SC,7.10,164.0,0.04,0.73,43.60,XR,ATH,FINANCE


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.50,7.14,60.78,72.27,110345.0,12100.0,181548.0,-7.89,74.0,M-LC,2.79,99.0,0.11,1.34,13.76,XR,NTT,IT
50,MASFIN,398.61,2.71,0.83,21.05,22.05,20795.0,810.0,98790.0,-13.28,63.0,H-SC,7.10,164.0,0.04,0.73,43.60,XR,ATH,FINANCE
38,INDIAMART,4810.62,-1.12,-1.74,110.38,106.72,133770.0,-2146.0,121190.0,-51.39,36.0,H-SC,1.34,139.0,-0.02,0.89,21.23,AR,ATH,MISC
37,IEX,219.00,0.19,-1.19,50.42,48.63,99613.0,-2388.0,197566.0,-34.34,57.0,H-SC,14.08,136.0,-0.02,1.46,10.22,XR,NTT,MISC
85,ZYDUSLIFE,1286.17,0.37,-1.41,37.57,35.63,77625.0,-2961.0,206613.0,-15.41,43.0,H-MC,3.15,119.0,-0.04,1.52,15.02,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.79,-21.85,89.95,48.45,140953.0,-43804.0,156702.0,-27.39,12.0,X-MC,10.02,64.0,-0.31,1.16,0.00,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-0.82,-38.14,118.55,35.20,94367.0,-49069.0,79601.0,4.40,25.0,X-SC,15.00,92.0,-0.52,0.59,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,0.21,-11.51,37.41,21.58,46389.0,-16137.0,124002.0,-55.40,27.0,X-MC,5.32,56.0,-0.35,0.91,9.78,X40N,ATH,IT
54,PGHH,17907.65,-0.57,-6.52,43.09,33.76,80891.0,-13095.0,187725.0,-34.12,27.0,X-MC,5.08,57.0,-0.16,1.38,0.00,X40,ATH,FMCG
75,TMPV,600.00,-0.33,-16.48,70.24,42.18,162153.0,-45555.0,230855.0,-25.52,29.0,X-LC,4.48,3.0,-0.28,1.70,0.00,XY24,NTT,AUTO


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.0,-0.81,-20.64,93.42,53.50,223709.0,-62285.0,239466.0,-61.59,34.0,X-MC,1.36,58.0,-0.28,1.77,6.36,XY24,NTT,FMCG
43,ITC,452.0,-0.20,-1.25,11.84,10.44,23399.0,-2509.0,197629.0,-41.15,48.0,X-LC,1.57,5.0,-0.11,1.46,4.24,X40,NTT,FMCG
3,ACC,3906.0,0.52,-24.04,116.10,64.15,209851.0,-57201.0,180750.0,-55.44,41.0,X-SC,1.65,82.0,-0.27,1.33,1.39,XY24,BTT,CEMENT
15,CAMS,4762.0,-1.83,-80.04,525.34,24.79,264031.0,-201597.0,50259.0,-80.87,44.0,X-SC,2.15,86.0,-0.76,0.37,22.86,X40N,NTT,MISC
36,ICICIPRULI,790.0,0.99,4.82,24.95,30.97,46220.0,8511.0,185249.0,-19.89,61.0,X-MC,2.24,75.0,0.18,1.37,17.98,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.82,-38.14,118.55,35.20,94367.0,-49069.0,79601.0,4.40,25.0,X-SC,15.00,92.0,-0.52,0.59,0.0,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-0.79,-21.85,89.95,48.45,140953.0,-43804.0,156702.0,-27.39,12.0,X-MC,10.02,64.0,-0.31,1.16,0.0,X40N,ATH,FINANCE
54,PGHH,17907.65,-0.57,-6.52,43.09,33.76,80891.0,-13095.0,187725.0,-34.12,27.0,X-MC,5.08,57.0,-0.16,1.38,0.0,X40,ATH,FMCG
55,QUESS,424.00,-1.27,-32.87,112.78,42.84,49217.0,-21366.0,43640.0,-55.92,32.0,X-SC,37.32,83.0,-0.43,0.32,0.0,XY24,NTT,MISC
75,TMPV,600.00,-0.33,-16.48,70.24,42.18,162153.0,-45555.0,230855.0,-25.52,29.0,X-LC,4.48,3.0,-0.28,1.70,0.0,XY24,NTT,AUTO


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-1.27,-32.87,112.78,42.84,49217.0,-21366.0,43640.0,-55.92,32.0,X-SC,37.32,83.0,-0.43,0.32,0.00,XY24,NTT,MISC
15,CAMS,4762.00,-1.83,-80.04,525.34,24.79,264031.0,-201597.0,50259.0,-80.87,44.0,X-SC,2.15,86.0,-0.76,0.37,22.86,X40N,NTT,MISC
59,RELAXO,1176.00,-0.32,-47.78,191.67,52.31,145290.0,-69358.0,75802.0,-44.44,34.0,X-SC,5.71,91.0,-0.48,0.56,1.46,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.82,-38.14,118.55,35.20,94367.0,-49069.0,79601.0,4.40,25.0,X-SC,15.00,92.0,-0.52,0.59,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,0.21,-11.51,37.41,21.58,46389.0,-16137.0,124002.0,-55.40,27.0,X-MC,5.32,56.0,-0.35,0.91,9.78,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.34,-11.71,35.11,19.28,107238.0,-40522.0,305434.0,-22.98,71.0,X-LC,6.04,1.0,-0.38,2.25,12.91,X40,ATH,IT
41,INFY,2275.00,0.43,10.50,40.16,54.87,138850.0,32854.0,345742.0,-13.85,71.0,X-LC,3.41,2.0,0.24,2.55,19.65,X40,BTT,IT
75,TMPV,600.00,-0.33,-16.48,70.24,42.18,162153.0,-45555.0,230855.0,-25.52,29.0,X-LC,4.48,3.0,-0.28,1.70,0.00,XY24,NTT,AUTO
81,VBL,671.64,-1.35,-4.36,41.86,35.67,126443.0,-13781.0,302061.0,-15.77,54.0,X-LC,4.91,4.0,-0.11,2.23,8.89,X40N,ATH,FMCG
43,ITC,452.00,-0.20,-1.25,11.84,10.44,23399.0,-2509.0,197629.0,-41.15,48.0,X-LC,1.57,5.0,-0.11,1.46,4.24,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.99,-36.40,108.95,32.89,53498.0,-28104.0,49103.0,-706.73,62.0,L-MC,6.44,259.0,-0.53,0.36,35.22,XR,NTT,BANKS
6,ASIANTILES,137.00,-1.57,-18.51,122.95,81.67,93988.0,-17366.0,76444.0,6866.67,34.0,L-SC,15.54,271.0,-0.18,0.56,48.04,XR,NTT,CERAMICS
50,MASFIN,398.61,2.71,0.83,21.05,22.05,20795.0,810.0,98790.0,-13.28,63.0,H-SC,7.10,164.0,0.04,0.73,43.60,XR,ATH,FINANCE
13,BSOFT,831.70,1.31,-20.52,89.71,50.79,100289.0,-28861.0,111792.0,2.05,74.0,H-SC,5.64,151.0,-0.29,0.82,30.16,XR,ATH,IT
39,INDIGOPNTS,1408.00,-0.30,-12.50,14.30,0.02,21842.0,-21816.0,152743.0,132.70,52.0,M-SC,16.87,234.0,-1.00,1.13,32.86,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,1.31,-20.52,89.71,50.79,100289.0,-28861.0,111792.0,2.05,74.0,H-SC,5.64,151.0,-0.29,0.82,30.16,XR,ATH,IT
5,ASIANPAINT,3460.25,-0.92,2.81,17.64,20.95,45658.0,7066.0,258834.0,1.76,73.0,X-LC,18.89,31.0,0.15,1.91,39.44,X40,ATH,PAINTS
32,HCLTECH,1922.01,0.48,9.74,13.66,24.73,36266.0,23563.0,265487.0,14.46,72.0,X-LC,5.55,7.0,0.65,1.96,26.46,X40,ATH,IT
84,WIPRO,420.00,0.50,7.14,60.78,72.27,110345.0,12100.0,181548.0,-7.89,74.0,M-LC,2.79,99.0,0.11,1.34,13.76,XR,NTT,IT
65,SIEMENS,4671.50,-0.64,-11.21,41.36,25.51,68339.0,-20865.0,165230.0,1.35,58.0,H-LC,3.86,49.0,-0.31,1.22,21.76,AR,ATH,ELECTRICAL


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.85
1,25,45.75
2,50,76.66


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.90
MC    30.17
LC    24.85
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.30
X40      22.49
X40N     10.87
XR       10.03
AR        9.38
XY25      9.34
OX40N     7.73
SR        1.01
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.47
X-MC    23.45
X-LC    21.26
M-SC    12.13
X-SC     6.93
H-MC     4.95
M-MC     1.41
L-SC     1.37
M-LC     1.34
H-LC     1.22
L-LC     1.03
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.35,-6.94,41.52
IT,12.88,-15.22,77.51
FINANCE,9.61,-19.10,69.43
ELECTRICAL,6.26,-10.55,51.04
PAINTS,6.17,-10.42,26.82
MISC,6.13,-56.14,120.10
INSURANCE,4.59,-1.83,37.71
PHARMA,4.07,-1.02,33.35
AUTO,3.46,-22.41,74.90


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3199055.0,21
XR,1300365.0,13
AR,1297585.0,10
X40,989087.0,14
X40N,975464.0,9
OX40N,742122.0,10
XY25,374935.0,6
SR,283132.0,2
MH,79329.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3655758.0,25
M-SC,1415118.0,15
X-MC,1302461.0,16
X-SC,980814.0,8
X-LC,879994.0,11
H-MC,406074.0,3
L-SC,273906.0,3
M-LC,110345.0,1
H-LC,68339.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1283672.0      6
           AR         891901.0      5
M-SC       XY24       803264.0      6
H-SC       XR         776915.0      7
X-SC       X40N       503531.0      3
X-MC       X40        454149.0      7
           XY24       388803.0      3
X-LC       X40        351411.0      5
H-SC       OX40N      340809.0      4
M-SC       OX40N      313438.0      5
X-SC       XY24       293756.0      3
H-SC       SR         283132.0      2
X-MC       X40N       271617.0      4
X-LC       XY24       235991.0      2
H-MC       AR         212505.0      2
X-LC       X40N       200316.0      2
H-MC       XY24       193569.0      1
X-MC       XY25       187892.0      2
L-SC       XR         186031.0      2
X-SC       X40        183527.0      2
M-SC       XR         173576.0      2
           AR         124840.0      2
M-LC       XR         110345.0      1
X-LC       XY25        92276.0      2
L-SC       OX40N       87875.0      1
H-SC       MH          79329.0      1
H-LC       AR          68339.0      1
M-MC       XY25        54774.0      1
L-MC       XR          53498.0      1
L-LC       XY25        39993.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 57.0 seconds
